In [ ]:
from flask import Flask, request, render_template, redirect, url_for, session
import mysql.connector

app = Flask(__name__)
app.secret_key = 'your key'

db_config = {
    'user': 'root',
    'password': 'your password',
    'host': 'localhost',
    'database': 'sakila'
}

@app.route('/', methods=['GET', 'POST'])
def index():
    conn = mysql.connector.connect(**db_config)
    cursor = conn.cursor()

    # Initialize or retrieve the last seen timestamp from the session
    last_seen_timestamp = session.get('last_seen_timestamp', '1970-01-01 00:00:00')

    if request.method == 'POST':
        # Get form inputs
        address = request.form['address']
        address2 = request.form['address2']
        district = request.form['district']
        city_id = request.form['city_id']
        postal_code = request.form['postal_code']
        phone = request.form['phone']
        location = request.form.get('location', None)

        # Insert the new record into the database
        insert_query = """
            INSERT INTO address (address, address2, district, city_id, postal_code, phone, location, last_update)
            VALUES (%s, %s, %s, %s, %s, %s, ST_GeomFromText(%s), NOW())
        """
        cursor.execute(insert_query, (address, address2, district, city_id, postal_code, phone, 'POINT(0 0)' if not location else location))
        conn.commit()

        # Update the session with the current timestamp (last update)
        session['last_seen_timestamp'] = last_seen_timestamp = cursor.lastrowid

    # Query for newly added records since the last seen timestamp
    select_query = """
        SELECT address, address2, district, city_id, postal_code, phone, last_update
        FROM address
        WHERE last_update > %s
        ORDER BY last_update DESC
    """
    cursor.execute(select_query, (last_seen_timestamp,))
    new_addresses = cursor.fetchall()

    cursor.close()
    conn.close()

    return render_template('index1.html', addresses=new_addresses)


if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Sep/2024 09:56:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Sep/2024 09:56:07] "POST / HTTP/1.1" 200 -
